In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
dataset1=pd.read_csv("prep.csv",index_col=None)

df2=dataset1

df = pd.get_dummies(df2, drop_first=True)
df=df.astype(int)

indep_X=df.drop('classification_yes',axis=1)
dep_Y=df['classification_yes']

In [3]:
df

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2,76,3,0,148,57,3,137,4,12,...,0,0,0,0,0,0,1,1,0,1
1,3,76,2,0,148,22,0,137,4,10,...,1,0,0,0,0,0,1,0,0,1
2,4,76,1,0,99,23,0,138,4,12,...,1,0,0,0,0,0,1,0,0,1
3,5,76,1,0,148,16,0,138,3,8,...,1,0,0,0,0,0,1,0,1,1
4,5,50,0,0,148,25,0,137,4,11,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51,70,0,0,219,36,1,139,3,12,...,1,0,0,0,0,0,1,0,0,1
395,51,70,0,2,220,68,2,137,4,8,...,1,0,0,1,1,0,1,0,1,1
396,51,70,3,0,110,115,6,134,2,9,...,1,0,0,1,1,0,0,0,0,1
397,51,90,0,0,207,80,6,142,5,8,...,1,0,0,1,1,0,1,0,1,1


In [4]:
def split_scalar(indep_X,dep_Y):
        X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size = 0.25, random_state = 0)
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)    
        return X_train, X_test, y_train, y_test, sc

def LDA(X_train,y_train,  X_test, n):
    from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
    lda = LDA(n_components=n)
    X_train1 =lda.fit_transform(X_train, y_train )
    X_test1= lda.transform(X_test)
    explained_variance = lda.explained_variance_ratio_
    print("lda:",lda)
    print("explained_variance:",explained_variance)
    return X_train1, X_test1, lda
    
def cm_prediction(classifier,X_test1):
        y_pred = classifier.predict(X_test1)
        
        # Making the Confusion Matrix
        from sklearn.metrics import confusion_matrix
        cm = confusion_matrix(y_test, y_pred)
        
        from sklearn.metrics import accuracy_score 
        from sklearn.metrics import classification_report 
        
        Accuracy=accuracy_score(y_test, y_pred)
        report=classification_report(y_test, y_pred)
        
        return  classifier,Accuracy,report,X_test1,y_test,cm

def logistic(X_train1,y_train,X_test1):
        from sklearn.linear_model import LogisticRegression
        classifier = LogisticRegression(random_state = 0)
        classifier.fit(X_train1, y_train)
        classifier,Accuracy,report,X_test1,y_test,cm=cm_prediction(classifier,X_test1)
        return  classifier,Accuracy,report,X_test1,y_test,cm      
    

def selectk_Classification(acclog): 
    
    dataframe=pd.DataFrame(index=['LDA'],columns=['Logistic'])
    for number,idex in enumerate(dataframe.index):      
        dataframe['Logistic'][idex]=acclog[number]       
    return dataframe

In [5]:
X_train, X_test, y_train, y_test, sc = split_scalar(indep_X,dep_Y)

In [6]:
print (" before LDA: ",X_train.shape, X_test.shape)

 before LDA:  (299, 27) (100, 27)


In [7]:
X_train1,X_test1, lda = LDA(X_train, y_train, X_test, 1)

acclog=[]

print (" After LDA: ", X_train1.shape, X_test1.shape)

lda: LinearDiscriminantAnalysis(n_components=1)
explained_variance: [1.]
 After LDA:  (299, 1) (100, 1)


In [8]:
classifier,Accuracy,report,X_test,y_test,cm=logistic(X_train1,y_train,X_test1)
acclog.append(Accuracy)
    
result=selectk_Classification(acclog)

In [9]:
cm

array([[36,  0],
       [ 2, 62]], dtype=int64)

In [10]:
Accuracy

0.98

In [11]:
report

'              precision    recall  f1-score   support\n\n           0       0.95      1.00      0.97        36\n           1       1.00      0.97      0.98        64\n\n    accuracy                           0.98       100\n   macro avg       0.97      0.98      0.98       100\nweighted avg       0.98      0.98      0.98       100\n'

In [12]:
result
#1

,Logistic
LDA,0.98


In [13]:
import pickle
scaler_file="scaler_x.pkl"
pickle.dump(sc,open(scaler_file,"wb"))
SCX=pickle.load(open("scaler_x.pkl",'rb'))

lda_file="lda.pkl"
pickle.dump(lda,open(lda_file,"wb"))
lda=pickle.load(open("lda.pkl",'rb'))

In [14]:
preinput=SCX.transform([[2,76.4,3,0,148,57,3,137,4.6,12.5,38.86,8408,4.70,0,1,0,0,1,0,0,0,0,0,0,1,1,0]])
preinput

C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[-3.10684855, -0.00502986,  1.75526219, -0.36317599, -0.00384577,
        -0.06333607,  0.03500082, -0.04166869,  0.25245102,  0.17039357,
         0.01224925,  0.06180541,  0.65415605, -0.48005487,  1.91621371,
        -0.46416285, -0.13041013,  0.38729833, -2.01687794, -0.32773069,
        -0.23777532, -0.74109567, -0.73029674, -0.29541958,  0.51147177,
         1.9551421 , -0.43730431]])

In [15]:
preinput1 = lda.transform(preinput)
preinput1

array([[-0.35485954]])

In [16]:
import pickle
filename="Finalized_model_LR_LDA.sav"

In [17]:
pickle.dump(classifier,open(filename,'wb'))

In [18]:
loaded_module=pickle.load(open("Finalized_model_LR_LDA.sav",'rb'))

In [19]:
result1=loaded_module.predict(preinput1)
result1

array([1])